# Bonding and topology determination in the Materials Project database



In [1]:
import json
import os
import random
import warnings

import juliacall

import pymatgen
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy, MultiWeightsChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter

from tqdm.notebook import tqdm

random.seed("Call me Ishmael")
print("Using pymatgen version:", pymatgen.core.__version__)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Using pymatgen version: 2025.2.18


In [2]:
jl = juliacall.newmodule("NotebookModule")
jl.seval("using CrystalNets")
jl.CrystalNets.toggle_export(False)

print("Using Julia version:", jl.seval("VERSION"))
print("Running from directory:", jl.seval("Sys.BINDIR"))

Using Julia version: 1.11.3
Running from directory: /opt/julia-1.11.3/bin


In [3]:
with open(os.path.expanduser("~/.mpapikey"), "r") as f:
    apikey = f.read().strip()

In [4]:
try:
    os.mkdir("cif")
except FileExistsError as e:
    pass

try:
    os.mkdir("cif_chemenv")
except FileExistsError as e:
    pass

In [5]:
crystallnn = CrystalNN()

chemenv_strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)
# We cannot use the more powerful multi-weight strategy, because it creates fractional coordination environment
# chemenv_strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()


def make_labels_unique(struct):
    from collections import Counter
    
    labels = Counter(site.label for site in struct.sites)
    counter = {}
    for i, site in enumerate(struct.sites):
        label = site.label
        if labels[label] > 1 or label.isalpha():
            c = counter.get(label, 0)
            site.label = f"{label}{c}" if label.isalpha() else f"{label}_{c}"
            c = c + 1
            counter[label] = c


def jimage_to_site_symmetry(jimage):
    i, j, k = jimage
    return f"1_{5+i}{5+j}{5+k}"


def writeToCifFile(bonded_struct, file):
    cif_writer = CifWriter(bonded_struct.structure)
    cif = str(cif_writer)
    cif += """loop_
_geom_bond_atom_site_label_1
_geom_bond_atom_site_label_2
_geom_bond_distance
_geom_bond_site_symmetry_2
"""

    for n, site in enumerate(bonded_struct.structure.sites):
        for connected in bonded_struct.get_connected_sites(n):
            # Make sure we only output each bond once
            if site.label <= connected.site.label:
                cif += f"{site.label} {connected.site.label} {connected.dist:.6f} {jimage_to_site_symmetry(connected.jimage)}\n"

    file.write(cif)


def bonded_structure_from_lse(lse: LightStructureEnvironments) -> StructureGraph:
    """
    Convert a LightStructureEnvironments from chemenv to a StructureGraph
    """
    graph = StructureGraph.from_empty_graph(lse.structure, name="bonds")

    for nset in lse.neighbors_sets:
        assert len(nset) == 1
        for neighbor in nset[0].neighb_indices_and_images:
            graph.add_edge(
                from_index=nset[0].isite,
                from_jimage=(0, 0, 0),
                to_index=neighbor["index"],
                to_jimage=neighbor["image_cell"],
                weight=1,
                edge_properties=None,
                warn_duplicates=False,
            )

    graph.set_node_attributes()
    return graph


def analyse_bonding(mat, method = "CrystalNN"):
    if method != "CrystalNN" and method != "ChemEnv":
        raise ValueError("invalid method name")

    # Magnetic moments trigger a bug in CifWriter, so we remove them here
    # https://github.com/materialsproject/pymatgen/issues/3772
    if "magmom" in mat.structure.site_properties:
        mat.structure.remove_site_property("magmom")

    # All labels should be unique (otherwise bond specifications will fail)
    make_labels_unique(mat.structure)
    labels = [site.label for site in mat.structure.sites]
    if len(labels) != len(set(labels)):
        raise ValueError("labels are not unique in structure")

    while True:
        bonded_struct = None
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            try:
                if method == "ChemEnv":
                    lgf = LocalGeometryFinder()
                    lgf.setup_structure(structure=mat.structure)
                    se = lgf.compute_structure_environments(
                        maximum_distance_factor=1.41, only_cations=False
                    )
                    lse = LightStructureEnvironments.from_structure_environments(
                        strategy=chemenv_strategy, structure_environments=se
                    )
                    bonded_struct = bonded_structure_from_lse(lse)
                else:
                    bonded_struct = crystallnn.get_bonded_structure(mat.structure)

            except Exception as e:
                print(f"{method} failed for {mat.material_id}: {e}")
                return None

        coord = set()
        bonds = set()
        rerun_flag = False
        for n, site in enumerate(bonded_struct.structure.sites):
            neighbors = bonded_struct.get_connected_sites(n)
            # We cannot use get_coordination_of_site() because of a bug:
            # https://github.com/materialsproject/pymatgen/issues/3888#issuecomment-2232571072
            coord.add((str(site.specie), len(neighbors)))
            for connected in neighbors:
                if site.specie <= connected.site.specie:
                    bonds.add((str(site.specie), str(connected.site.specie)))
                if site.label == connected.site.label:
                    # If an atom is bonded to its own image, then CrystalNets will not like it
                    # and we need to make rerun on a supercell
                    rerun_flag = True

        if rerun_flag:
            # If we found an atom bonded to itself, 
            mat.structure.make_supercell(2, in_place=True)
            make_labels_unique(mat.structure)
        else:
            # Otherwise we're done
            break

    # Write bonded structure as CIF; output directory depends on method
    if method == "ChemEnv":
        dirname = "cif_chemenv"
    else:
        dirname = "cif"
    with open(f"{dirname}/{mat.material_id}.cif", "w") as f:
        writeToCifFile(bonded_struct, f)

    return {"material_id": str(mat.material_id),
            "formula_pretty": mat.formula_pretty,
            "nelements": mat.nelements,
            "theoretical": mat.theoretical,
            "is_stable": mat.is_stable,
            "crystal_system": str(mat.symmetry.crystal_system),
            "space_group": mat.symmetry.number,
            "coordination": coord,
            "bonds": bonds}

In [6]:
# Test our analysis method on one material
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        material_ids=["mp-3934"],
        fields=["material_id", "builder_meta", "deprecated", "formula_pretty", "nelements", "structure", "theoretical", "symmetry"]
    )

    assert len(mp_data) == 1
    res = analyse_bonding(mp_data[0])
    print(res)

    res = analyse_bonding(mp_data[0], "ChemEnv")
    print(res)

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

{'material_id': 'mp-3934', 'formula_pretty': 'Cu3PS4', 'nelements': 3, 'theoretical': False, 'is_stable': None, 'crystal_system': 'Orthorhombic', 'space_group': 31, 'coordination': {('P', 4), ('S', 4), ('Cu', 4)}, 'bonds': {('Cu', 'S'), ('P', 'S')}}
{'material_id': 'mp-3934', 'formula_pretty': 'Cu3PS4', 'nelements': 3, 'theoretical': False, 'is_stable': None, 'crystal_system': 'Orthorhombic', 'space_group': 31, 'coordination': {('P', 4), ('S', 4), ('Cu', 4)}, 'bonds': {('Cu', 'S'), ('P', 'S')}}


In [7]:
with MPRester(apikey) as mpr:
    mp_data = mpr.materials.summary.search(
        deprecated=False,
        fields=["material_id", "builder_meta", "deprecated", "formula_pretty", "nelements", "structure", "theoretical", "symmetry"]
    )

print("Database version", mpr.get_database_version())
print("Number of materials found:", len(mp_data))

# Check that we did not get any deprecated material
# See https://github.com/materialsproject/api/issues/964
assert sum(1 for x in mp_data if x.deprecated) == 0

Retrieving SummaryDoc documents:   0%|          | 0/170470 [00:00<?, ?it/s]

Database version 2025.02.12.post
Number of materials found: 170470


In [8]:
n_gnome = sum(1 for x in mp_data if x.builder_meta.batch_id and "gnome" in x.builder_meta.batch_id)
n_ccbynd = sum(1 for x in mp_data if x.builder_meta.license == "BY-NC")

print("Number of GnoME structures:", n_gnome)
print("Number of non-GnoME structures:", len(mp_data) - n_gnome)
print("Number of CC-BY-ND structures:", n_ccbynd)

Number of GnoME structures: 15483
Number of non-GnoME structures: 154987
Number of CC-BY-ND structures: 15483


In [9]:
data = {}

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="No Pauling electronegativity")
    for mat in tqdm(mp_data, smoothing=0.01, mininterval=1):
        x = analyse_bonding(mat)
        if x is not None:
            data[mat.material_id] = x

print("Number of systems analyzed:", len(data))

  0%|          | 0/170470 [00:00<?, ?it/s]

CrystalNN failed for mp-994911: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1210439: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1247838: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1213668: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1215144: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1180797: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1247813: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1212347: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1214815: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1215160: No Voronoi neighbors found for site - try increasing cutoff
CrystalNN failed for mp-1180180: No Voronoi neighbors found for site - 

In [10]:
sample_data = random.sample(mp_data, 5000)
data_chemenv = {}

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="No Pauling electronegativity")
    for mat in tqdm(sample_data, smoothing=0.01, mininterval=1):
        x = analyse_bonding(mat, "ChemEnv")
        if x is not None:
            data_chemenv[mat.material_id] = x

print("Number of systems analyzed:", len(data_chemenv))

  0%|          | 0/5000 [00:00<?, ?it/s]

ChemEnv failed for mp-1093836: QH6239 Qhull precision error: initial Delaunay input sites are cocircular or cospherical.  Use option 'Qz' for the Delaunay triangulation or Voronoi diagram of cocircular/cospherical points; it adds a point "at infinity".  Alternatively use option 'QJ' to joggle the input.  Use option 'Qs' to search all points for the initial simplex.

input sites with last coordinate projected to a paraboloid
- p3(v5):  -2.2 5.7e-17 1.2e-16   4.8
- p0(v4):   2.2 -5.7e-17 -1.2e-16   4.8
- p4(v3):   8.1 2.8e-16 -3.3e-16    65
- p6(v2):    10     0     0 1.1e+02
- p5(v1):  -5.1     0     0    26

While executing:  | qhull v o Fv
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1858205819  voronoi  offFile  _pre-merge  _zero-centrum  Pgood
  Fvoronoi  _max-width 1.1e+02  Error-roundoff 1.3e-13  _one-merge 1.1e-12
  _near-inside 5.7e-12  Visible-distance 7.7e-13  U-max-coplanar 7.7e-13
  Width-outside 1.5e-12  _wide-facet 4.6e-12  _maxoutside 1.5e-12

ChemEnv failed f

In [11]:
# Number of systems where CrystalNN and ChemEnv disagree
sum(1 for k in data_chemenv.keys() if data_chemenv[k] != data.get(k, None))

3114

In [12]:
crystalnets_options = jl.CrystalNets.Options(structure=jl.StructureType.Auto, clusterings=[jl.Clustering.EachVertex], bonding=jl.Bonding.Input, split_O_vertex=False)

def analyse_topology(material_id):
    res = jl.determine_topology(f"cif/{material_id}.cif", crystalnets_options)
    topologies = [(mult, jl.ndims(net[jl.Clustering.EachVertex].genome), str(net[jl.Clustering.EachVertex])) for net, mult in res]
    return topologies

In [13]:
analyse_topology("mp-1206677")

[(1, 3, 'pcu')]

In [14]:
jl.CrystalNets.toggle_export(False)
jl.CrystalNets.toggle_warning(False)

for material_id in tqdm(data.keys(), smoothing=0.01, mininterval=1):
    topo = analyse_topology(material_id)
    data[material_id]["topology"] = topo

  0%|          | 0/170455 [00:00<?, ?it/s]

spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).
spglib: Indicated max size(=384) is less than number spglib: of symmetry operations(=768).


In [15]:
data["mp-1206677"]

{'material_id': 'mp-1206677',
 'formula_pretty': 'Rb2O2',
 'nelements': 2,
 'theoretical': False,
 'is_stable': None,
 'crystal_system': 'Monoclinic',
 'space_group': 11,
 'coordination': {('O', 6), ('Rb', 6)},
 'bonds': {('Rb', 'O')},
 'topology': [(1, 3, 'pcu')]}

In [16]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

with open("topo_data.json", "w") as file:
    json.dump(data, file, cls=SetEncoder)

In [17]:
# Compare with topology from ChemEnv bonding

crystalnets_options = jl.CrystalNets.Options(structure=jl.StructureType.Auto, clusterings=[jl.Clustering.EachVertex], bonding=jl.Bonding.Input, split_O_vertex=False)

def analyse_topology_chemenv(material_id):
    res = jl.determine_topology(f"cif_chemenv/{material_id}.cif", crystalnets_options)
    topologies = [(mult, jl.ndims(net[jl.Clustering.EachVertex].genome), str(net[jl.Clustering.EachVertex])) for net, mult in res]
    return topologies

for material_id in tqdm(data_chemenv.keys(), smoothing=0.01, mininterval=1):
    topo = analyse_topology_chemenv(material_id)
    data_chemenv[material_id]["topology"] = topo

  0%|          | 0/4931 [00:00<?, ?it/s]

In [19]:
# Number of systems where CrystalNN and ChemEnv disagree
sum(1 for k in data_chemenv.keys() if data_chemenv[k]["topology"] != data.get(k, {}).get("topology", None))

3091

In [20]:
sum(1 for k in data_chemenv.keys() if data_chemenv[k] != data.get(k, None))

3147

In [29]:
list(str(k) for k in data_chemenv.keys() if data_chemenv[k] != data.get(k, None) and 'UNKNOWN' not in data_chemenv[k]['topology'][0][2])[:10]

['mp-978276',
 'mp-1187858',
 'mp-1220734',
 'mp-1182251',
 'mp-1217922',
 'mp-2458',
 'mp-276',
 'mp-1179374',
 'mp-1226378',
 'mp-9472']

In [32]:
data_chemenv['mp-978276']

{'material_id': 'mp-978276',
 'formula_pretty': 'Mg3Co',
 'nelements': 2,
 'theoretical': True,
 'is_stable': None,
 'crystal_system': 'Cubic',
 'space_group': 225,
 'coordination': {('Co', 14), ('Mg', 14)},
 'bonds': {('Mg', 'Co'), ('Mg', 'Mg')},
 'topology': [(1, 3, 'bcu-x')]}

In [33]:
data['mp-978276']

{'material_id': 'mp-978276',
 'formula_pretty': 'Mg3Co',
 'nelements': 2,
 'theoretical': True,
 'is_stable': None,
 'crystal_system': 'Cubic',
 'space_group': 225,
 'coordination': {('Co', 14), ('Mg', 8), ('Mg', 14)},
 'bonds': {('Mg', 'Co'), ('Mg', 'Mg')},
 'topology': [(1, 3, 'reo-d')]}

In [ ]:
with open("topo_data_chemenv.json", "w") as file:
    json.dump(data_chemenv, file, cls=SetEncoder)